In [143]:
### import required libraries

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import cfscrape

In [2]:
### set date range for weekly report
date_range_high = datetime.today().date()
date_range_low = datetime.today().date() - timedelta(days=7)

In [3]:
user_agent = 'Mozilla/5.0 (iPad; U; CPU OS 3_2_1 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Mobile/7B405'

#### Reuters M&A News

In [4]:
### webscrape Reuters M&A News

date_filter = date_range_high
page_number = 1
reuters_news_dict = {}
reuters_news_url = "https://www.reuters.com/news/archive/mergersnews?view=page&page={}&pageSize=10"

while date_filter >= date_range_low:

    reuters_raw = requests.get(reuters_news_url.format(page_number), headers={'User-Agent': user_agent})
    reuters_bs4 = BeautifulSoup(reuters_raw.content)

    reuters_news_block = reuters_bs4.find_all('div', class_ = 'column1 col col-10')
    reuters_news_list  = reuters_news_block[0].find_all('h3', class_ = 'story-title')
    reuters_timestamp_list = reuters_news_block[0].find_all('span', class_ = 'timestamp')

    for i in range(len(reuters_news_list)):
        title = reuters_news_list[i].get_text()
        title = title.split('\n\t\t\t\t\t\t\t\t')[1]

        a_date = reuters_timestamp_list[i].get_text()
        if 'am' in a_date or 'pm' in a_date:
            a_date = datetime.today().date()
        else:
            a_date = datetime.strptime(a_date, "%b %d %Y").date()

        reuters_news_dict[title] = a_date

    date_filter = a_date
    page_number += 1

In [5]:
reuters_news_dict = {title: date for title, date in reuters_news_dict.items() if date >= date_range_low}

In [7]:
len(reuters_news_dict)

101

#### Seeking Alpha M&A News

In [149]:
### webscrape Seeking Alpha M&A News
options = webdriver.ChromeOptions()
options.add_argument("headless")

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://seekingalpha.com/market-news/m-a?page=1')



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/babyghost/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache
/var/folders/hg/yzr2mgl56dd1fz9lpflpnsmr0000gn/T/ipykernel_14327/2404279327.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [150]:
article_block = driver.find_elements(By.XPATH, "//*[@id='content']/div/div[2]/div/div[2]/section/div/div/div/div[2]")[0].text.split('\n')

article_list = []
for i, j in enumerate(article_block):
    if i%2 == 0:
        article_list.append(j)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=97.0.4692.71)
Stacktrace:
0   chromedriver                        0x000000010ef9ee69 chromedriver + 5160553
1   chromedriver                        0x000000010ef29593 chromedriver + 4679059
2   chromedriver                        0x000000010eadd2c8 chromedriver + 172744
3   chromedriver                        0x000000010eacca05 chromedriver + 104965
4   chromedriver                        0x000000010eacdeb2 chromedriver + 110258
5   chromedriver                        0x000000010eac6ba2 chromedriver + 80802
6   chromedriver                        0x000000010eade753 chromedriver + 178003
7   chromedriver                        0x000000010eb4275c chromedriver + 587612
8   chromedriver                        0x000000010eb30623 chromedriver + 513571
9   chromedriver                        0x000000010eb05dce chromedriver + 339406
10  chromedriver                        0x000000010eb07105 chrome

In [130]:
date_list = driver.find_elements(By.XPATH, "//span[@class = 'zmL zmF']")
date_list = [i.text for i in date_list] 

formatted_date_list = []

for a_date in date_list:
    if 'Today' in a_date:
        a_date = datetime.today().date()
    elif 'Yesterday' in a_date:
        a_date = datetime.today().date() - timedelta(days = 1)
    elif len(a_date.split(' ')) == 3:
        a_date = a_date + ' ' + str(datetime.today().date().year)
        a_date = datetime.strptime(a_date, "%a, %b. %d %Y").date()
    else:
        a_date = datetime.strptime(a_date, "%a, %b. %d, %Y").date()
    
    formatted_date_list.append(a_date)
        

In [131]:
seeking_alpha_news_dict = dict(zip(article_list, formatted_date_list))

In [ ]:
driver.close()

In [132]:
seeking_alpha_news_dict

{"FTC head Khan sees a 'fierce sense of urgency' to her job at the antitrust agency": datetime.date(2022, 1, 19),
 'DSS acquires 25.1% of Alset International': datetime.date(2022, 1, 19),
 'Nutrien may be BHP takeover target, Gordon Haskett analyst says': datetime.date(2022, 1, 19),
 "GlaxoSmithKline drops on report that Unilever won't increase bid for consumer unit above GBP50B": datetime.date(2022, 1, 19),
 'Qiagen ticks higher on renewed takeover speculation': datetime.date(2022, 1, 19),
 "Kohl's nears high on report of retaining Goldman after activist push": datetime.date(2022, 1, 19),
 "Xilinx gains on report of status update for China's review of the AMD deal": datetime.date(2022, 1, 19),
 'Autodesk acquires cloud-based dailies platform Moxion': datetime.date(2022, 1, 19),
 'Casper Sleep gains after shareholders approve sale to Durational': datetime.date(2022, 1, 19),
 'Bank First to acquire Denmark Bancshares for $119M': datetime.date(2022, 1, 19),
 'Ginkgo Bioworks acquires Bos

#### theMiddleMarket M&A News

In [166]:
ma_news_url = 'https://www.themiddlemarket.com/latest-news'
driver = webdriver.Chrome('/Users/babyghost/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver')
driver.get(ma_news_url)

/var/folders/hg/yzr2mgl56dd1fz9lpflpnsmr0000gn/T/ipykernel_14327/3739062475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/babyghost/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver')


In [167]:
latest_news = driver.find_elements(By.XPATH,"/html/body/main/div/div/div/div[1]/div/div")
latest_news = latest_news[0].text.split('\n')

ma_article_list = [i for count, i in enumerate(latest_news) if count%2 == 0 ] 
ma_date_list = [i for count, i in enumerate(latest_news) if count%2 != 0 ] 
theMiddleMarket_news_dict =  dict(zip(ma_article_list, ma_date_list))

In [192]:
theMiddleMarket_news_dict = {key: datetime.strptime(value.title(), '%B %d, %Y').date() for key, value in theMiddleMarket_news_dict.items()}
theMiddleMarket_news_dict

{'Babylon Purchases DayToDay Health': datetime.date(2022, 1, 19),
 'Condeco Acquires Proxyclick': datetime.date(2022, 1, 19),
 'Ten Coves Closes Third Fund': datetime.date(2022, 1, 19),
 'Sigma Defense Purchases Solute': datetime.date(2022, 1, 19),
 'Kent Outdoors Acquires Bote and Kona': datetime.date(2022, 1, 19),
 'Outdoor Living Makes Three Acquisitions': datetime.date(2022, 1, 19),
 'All Star Auto Acquires Autolights': datetime.date(2022, 1, 19),
 'Monomoy Closes its Oversubscribed Fund IV': datetime.date(2022, 1, 19),
 'MSD Partners Appoints Kevin Silva': datetime.date(2022, 1, 19),
 'PayrHealth Buys SilverCreek RCM': datetime.date(2022, 1, 19),
 'Kidd & Co. Leads Recapitalization of PW': datetime.date(2022, 1, 19),
 'Ara Partners Promotes Wang and Nommensen': datetime.date(2022, 1, 19),
 'Precisely Purchases PlaceIQ': datetime.date(2022, 1, 19),
 'Odyssey Investment Makes 9 Promotions': datetime.date(2022, 1, 19),
 'Timetoact Group Acquires PKS Software': datetime.date(2022, 1, 